In [51]:
import os
import numpy as np
import pronouncing
import re
import tensorflow as tf
import random

In [91]:
num = 2
DATA_PATH = './outputs/generated_output_{0}.txt'
data_file = open(DATA_PATH.format(num), "r").read().split()

In [92]:
def syllable_count(word):
    count = 0
    vowels = "aeiouy"
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)):
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
    if word.endswith("e"):
        count -= 1
    if count == 0:
        count += 1
    return count

In [93]:
line = ""
data_lines = []
for word in data_file:
    if (word != "'"):
        line += word + " "
    if (len(line) > 10 and syllable_count(line) >= 10):
        data_lines.append(line[:-1])
        line = ""

In [94]:
# rhyming scheme : aabb
poems_lines = [line.strip() for line in data_lines]

last_words = [re.sub(r'[^\w\s]', '', line.split(" ")[-1]) for line in poems_lines]

lines_dict = {idx:word for idx, word in enumerate(poems_lines)}
last_word_dict = {i:w for i, w in enumerate(last_words)}

In [95]:
rhymes_dict = {}
for i, word_0 in last_word_dict.items():
    potential_rhymes = pronouncing.rhymes(word_0)
    rhymes = []
    
    for j, word_1 in last_word_dict.items():
        if (word_0 != word_1):
            if (word_1 in potential_rhymes):
                rhymes.append(j)
    
    rhymes_dict[i] = rhymes

In [96]:
def generate_rhyming_lines(rhymes, num):
    while 1:
        i = random.randrange(len(rhymes.keys()))
        if len(rhymes[i]) >= num - 1:
            pool = rhymes[i] + [i]
            return random.sample(set(pool), num)

In [97]:
def paragraph_maker(rhymes):
    a = generate_rhyming_lines(rhymes, 2)
    b = generate_rhyming_lines(rhymes, 2)
    c = generate_rhyming_lines(rhymes, 2)
    couple_0 = lines_dict[a[0]] + "\n"  + lines_dict[a[1]] + "\n"
    couple_1 = lines_dict[b[0]] + "\n"  + lines_dict[b[1]] + "\n"
    couple_2 = lines_dict[c[0]] + "\n"  + lines_dict[c[1]] + "\n"
    return couple_0 + couple_1 + couple_2

In [98]:
def poem_maker(rhymes):
    first = paragraph_maker(rhymes)
    second = paragraph_maker(rhymes)
    third = paragraph_maker(rhymes)
    fourth = paragraph_maker(rhymes)
    return first + "\n \n" + second + "\n \n" + third

In [99]:
names = ["The Road \n\n", "My Life\n\n", "The Book\n\n"]
OUTPUT_POEMS = './outputs/generated_poems.txt'
generated_poems = open(OUTPUT_POEMS, 'a+')

poem = poem_maker(rhymes_dict)
generated_poems.write(names[num])
generated_poems.write(poem)
generated_poems.write("\n\n\n\n")

generated_poems.close()